<h1> Uygulamalı Veri Bilimi Ödevi Veri Analiz Bölümü </h1>

<h3>İsim: Vehbi Özcan</h3>
<h3>Numara: 211220070</h3>

## Import Bölümü

In [76]:
import pandas as pd
import numpy as np
#Preprocessing işlemleri için
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
#Eğitim verisini parçalamak için 
from sklearn.model_selection import train_test_split
#Model Kayıt İşlemleri ve Model Yüklemek İçin Joblib
from joblib import Parallel, delayed 
import joblib 
#XGBoost Classifier
import xgboost as xgb
#Accuracy Score Metrics
from sklearn.metrics import accuracy_score



## Veriyi Okuma

In [77]:
org_data_ = pd.read_excel('data.xlsx')
#data = org_data.copy()
#org_data.head()


In [78]:
df_ = org_data_.copy()
df_.head()

,Bellek Hızı,Cihaz Ağırlığı,Ekran Boyutu,Ekran Kartı Bellek Tipi,Ekran Kartı Hafızası,Ekran Kartı Tipi,Ekran Panel Tipi,İşlemci Nesli,İşlemci,İşletim Sistemi,Kart Okuyucu,Aydınlatma,Maksimum İşlemci Hızı,Max Ekran Çözünürlüğü,Parmak İzi Okuyucu,Ram (Sistem Belleği),SSD Kapasitesi,Ürün Modeli,Fiyat
0,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Çok Yüksek
1,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Çok Yüksek
2,1066 MHz,NaN,10 inç,NaN,1 GB,NaN,IPS,1.Nesil,1000M,Android,NaN,0,"1,05 GHz",NaN,NaN,NaN,1 TB,Notebook,Orta
3,3200 MHz,1 - 2 kg,"15,6 inç",GDDR4,2 GB,Harici Ekran Kartı,LED,10. Nesil,1035G1,Windows 10 Home,Var,0,"3,6 GHz",1920 x 1080,Yok,8 GB,512 GB,Notebook,Düşük
4,3200 MHz,1 - 2 kg,"15,6 inç",GDDR5,2 GB,Harici Ekran Kartı,LED,10. Nesil,1035G1,Windows 10 Home,Yok,0,"3,6 GHz",1920 x 1080,Yok,12 GB,1 TB,Notebook,Düşük


## Preprocessing

### Boş Verileri Doldurma Fonksiyonu

In [80]:
def fill_value(df):
    for column in df.columns:
        df[column].fillna(value = df[column].mode()[0], inplace=True)
    return df

### Aykırı ve Gürültülü Verileri Düzeltme Fonksiyonu


In [81]:
def fix_value(df):
    df['İşlemci'].replace('10. Nesil Intel® Core™ i3-10110U işlemci', '10110U',inplace=True)
    df['İşlemci Nesli'].replace('10. Nesil', '10.Nesil',inplace=True)
    df['SSD Kapasitesi'].replace(' 0 GB', '0 GB',inplace=True)
    df['Ram (Sistem Belleği)'].replace('8\xa0GB', '8 GB',inplace=True)
    return df

### Normalizasyon (MinMaxScaler) Fonksiyonu

In [82]:
def df_minmaxscaler(le_df):
    scaler = MinMaxScaler()
    le_dfr = np.reshape(le_df,(-1,1))
    le_dfr = le_dfr.astype(float)
    le_dfs = scaler.fit_transform(le_dfr)
    le_dfs = le_dfs.flatten()
    return le_dfs

### Encoding (One Hot Encoder ve LabelEncoder) Fonksiyonu

In [83]:
def df_encode(df):
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')
    le = LabelEncoder()
    columns = df.columns.to_list()
    columns.remove("Fiyat")
    encoded_df = ohe.fit_transform(df[columns])
    le_df = le.fit_transform(df["Fiyat"])
    #le_dfs = df_minmaxscaler(le_df)
    #encoded_df["Fiyat"] = le_dfs
    encoded_df["Fiyat"] = le_df
    return encoded_df

## Preprocessing Function

In [84]:
def preprocess(df):
    df = fill_value(df)
    df = fix_value(df)
    encoded_df = df_encode(df)
    return encoded_df

In [85]:
encoded_df_ = preprocess(df_)
encoded_df_


,Bellek Hızı_1066 MHz,Bellek Hızı_1600 MHz,Bellek Hızı_1866 MHz,Bellek Hızı_2133 MHz,Bellek Hızı_2400 MHz,Bellek Hızı_2666 MHz,Bellek Hızı_2933 MHz,Bellek Hızı_3000 MHz,Bellek Hızı_3200 MHz,Bellek Hızı_3733 MHz,...,SSD Kapasitesi_256 GB,SSD Kapasitesi_4 TB,SSD Kapasitesi_500 GB,SSD Kapasitesi_512 GB,SSD Kapasitesi_64 GB,Ürün Modeli_Notebook,Ürün Modeli_Oyun Bilgisayarları,Ürün Modeli_Ultrabook,Ürün Modeli_İkisi Bir Arada,Fiyat
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
991,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0


## Classification Function

In [86]:
def classification():
    org_data = pd.read_excel('data.xlsx')
    df = org_data.copy()

    encoded_df = preprocess(df)

    X = encoded_df.drop(["Fiyat"],axis=1) 
    y = encoded_df["Fiyat"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 60)

    xgb_cls = xgb.XGBClassifier(objective='multiclass:softmax', num_class=5)
    xgb_model = xgb_cls.fit(X_train,y_train)
    y_pred = xgb_model.predict(X_test)

    joblib.dump(xgb_model, 'modelxgb.pkl')
    score = accuracy_score(y_test,y_pred)
    print(score)
    return score



In [87]:
classification()

0.8844221105527639


0.8844221105527639